# Imports

In [3]:
from google.cloud import storage
import pandas as pd
import numpy as np
import re

# Carregando Dados

In [4]:
BUCKET_NAME = 'busca-apartamentos-bucket'
FILE = '2024-02-14 - apartamentos - cilar.csv'
TEMP_FILE = 'local.csv'

storage_client = storage.Client()
bucket = storage_client.get_bucket(BUCKET_NAME)

blop = bucket.blob(blob_name= FILE).download_as_string()
with open(TEMP_FILE, 'wb') as f:
    f.write(blop)

df = pd.read_csv('local.csv')

In [14]:
df

,titulo,link,endereco,detalhes,aluguel,condominio,iptu,catacteristicas_imovel,detalhes_condominio,mais_detalhes_imovel
0,Apartamento no 15º andar com 3 quartos,https://cilar.com.br/alugar/apartamento-no-15-...,"Rua Professor Pedro Viriato Parigot Souza, 306...","Área Total 421,62m2 Quartos 3 Suítes 3 Andar 15","AluguelR$ 13.000,00","Condominio R$ 3.189,70","IPTU R$ 838,00",1ª LOCAÇÃO Apartamento de alto padrão contend...,"Piscina, Academia, Salão de festas, Playground...","Andar: 15, Ano de construção: 2021, Aptos por ..."
1,Apartamento no 1º andar com 3 quartos,https://cilar.com.br/alugar/apartamento-no-1-a...,"Avenida Paraná, 111 - Cabral - Curitiba","Área Total 403,08m2 Quartos 3 Banheiros 1 Suít...","AluguelR$ 12.000,00","Condominio R$ 2.348,64","IPTU R$ 782,96",Apartamento mobiliado contendo: 1 suíte com cl...,"Elevador, Garagem Coberta","Andar: 01, Ano de construção: 1999, Aptos por ..."
2,Apartamento no 1º andar com 4 quartos,https://cilar.com.br/alugar/apartamento-no-1-a...,"Rua Deputado Joaquim José Pedrosa, 618 - Cabra...","Área Total 316,01m2 Quartos 4 Banheiros 1 Suít...","AluguelR$ 5.000,00","Condominio R$ 1.926,04","IPTU R$ 404,97",3 salas com teto rebaixado sendo 1 com sacada ...,"Salão de festas, Playground, Quadra de esporte...","Água: sim, Andar: 01, Ano de construção: 1979,..."
3,Apartamento no 11º andar com 4 quartos,https://cilar.com.br/alugar/apartamento-no-11-...,"Avenida Sete de Setembro, 4699 - Batel - Curitiba","Área Total 227,00m2 Quartos 4 Banheiros 1 Suít...","AluguelR$ 4.000,00","Condominio R$ 1.350,00","IPTU R$ 428,00",1 sala para 2 ambientes; 1 sala de jantar; 1 l...,"Academia, Salão de festas, Churrasqueira, Quad...","Andar: 11, Ano de construção: 1989, Aptos por ..."
4,Apartamento com 3 quartos,https://cilar.com.br/alugar/apartamento-com-3-...,"Avenida República Argentina, 1690 - Portão - C...","Área Total 169,62m2 Quartos 3 Banheiros 1 Suít...","AluguelR$ 3.900,00","Condominio R$ 804,60","IPTU R$ 158,00","1 cozinha com armário, pia tampo granito, 1 me...",Garagem Coberta,"Ano de construção: 1986, Aptos por andar: 4, Á..."
...,...,...,...,...,...,...,...,...,...,...
466,Apartamento no 2º andar com 2 quartos,https://cilar.com.br/alugar/apartamento-no-2-a...,"Gaipava, 211 - Pinheirinho - Curitiba","Área Total 0,00m2 Quartos 2 Banheiros 1 Andar 2","AluguelR$ 850,00",NaN,NaN,Imediações: Colégio Estadual Professora Etelvi...,NaN,"Andar: 2, Ano de construção: 2000, Área privat..."
467,Apartamento no 4º andar com 2 quartos,https://cilar.com.br/alugar/apartamento-no-4-a...,"R HOWELL LEWIS FRY, 341 - CACHOEIRA - Curitiba","Área Total 52,82m2 Quartos 2 Banheiros 1 Andar 4","AluguelR$ 850,00","Condominio R$ 320,00","IPTU R$ 40,48",Belo apartamento situado em uma área estratégi...,"Salão de festas, Playground, Portaria 24h, Gar...","Andar: 4, Aptos por andar: 4, Área privativa: ..."
468,Apartamento com 2 quartos,https://cilar.com.br/alugar/apartamento-com-2-...,"R Atilio Borio, 139 - Cristo Rei - Curitiba","Área Total 52,00m2 Quartos 2 Banheiros 1","AluguelR$ 850,00","Condominio R$ 465,00","IPTU R$ 50,79",Imediações: RODOFERROVIARIA Detalhes: 02 DOR...,Elevador,"Ano de construção: 33 Ano(s), Área privativa: ..."
469,Apartamento no 2º andar com 2 quartos,https://cilar.com.br/alugar/apartamento-no-2-a...,"Rua Rio Jaguaribe, 509 - Bairro Alto - Curitiba","Área Total 40,00m2 Quartos 2 Banheiros 1 Andar 2","AluguelR$ 800,00","Condominio R$ 350,00","IPTU R$ 28,00","Apartamento localizado no Bairro Alto, com 2 q...",Garagem Coberta,"Andar: 2, Área privativa: 39, Área total: 40, ..."


# Tratando Dados

## Limpeza

In [15]:
df['detalhes'] = df['detalhes'].apply(lambda x: x if isinstance(x,float) else x.replace("['", "").replace("']", "").replace("', '", ' ').replace('Características do imóvel ', ''))

df['iptu'] = df[['condominio','iptu']].apply(lambda x: 
                                             x['iptu'] if (isinstance(x['condominio'],float)) else 
                                             x['condominio'] if ('IPTU' in x['condominio']) else x['iptu'], axis = 1)

df['condominio'] = df[['condominio','iptu']].apply(lambda x: 
                                             x['condominio'] if (isinstance(x['condominio'],float)) else 
                                             np.nan if ('IPTU' in x['condominio']) else x['condominio'], axis = 1)

df['aluguel'] = df['aluguel'].apply(lambda x: x if isinstance(x,float) else int(x.replace('Aluguel','').replace('R$','').replace(',00','').replace('.','')))
df['condominio'] = df['condominio'].apply(lambda x: x if isinstance(x, float) else float(x.replace('Condominio  ','').replace('R$ ', '').replace('.','').replace(',','.')))
df['iptu'] = df['iptu'].apply(lambda x: x if isinstance(x, float) else float(x.replace('IPTU  ','').replace('R$ ','').replace('.',',').replace(',','.')))

# Criação de Features

In [16]:
def extrai_valores_string(string,substring):

    # Padronizar a expressão regular para encontrar a área total
    padrao = f'{substring} (\d+)'

    # Encontrar a área total usando regex
    area_total = re.search(padrao, string)

    if area_total:
        # Extrair o valor numérico da área total
        valor_area = area_total.group(1)
        
        # Remover vírgulas e converter para float
        valor_area = int(valor_area.replace(',', '.'))
        
    else:
        valor_area = 0
    
    return valor_area

In [17]:
df['detalhes'].apply(lambda x: np.nan if isinstance(x,float) else extrai_valores_string(x,'Área Total'))

0      421.0
1      403.0
2      316.0
3      227.0
4      169.0
       ...  
466      0.0
467     52.0
468     52.0
469     40.0
470     30.0
Name: detalhes, Length: 471, dtype: float64

In [20]:
## Detalhes do imóvel
df['area'] = df['detalhes'].apply(lambda x: x if isinstance(x,float) else extrai_valores_string(x,'Área Total'))
df['quartos'] = df['detalhes'].apply(lambda x: x if isinstance(x,float) else extrai_valores_string(x,'Quartos'))
df['suites'] = df['detalhes'].apply(lambda x: x if isinstance(x,float) else extrai_valores_string(x,'Suítes'))
df['banheiros'] = df['detalhes'].apply(lambda x: x if isinstance(x,float) else extrai_valores_string(x,'Banheiros'))
df['andar'] = df['detalhes'].apply(lambda x: x if isinstance(x,float) else extrai_valores_string(x,'Andar'))
df['vagas_garagem'] = df['detalhes'].apply(lambda x: x if isinstance(x,float) else extrai_valores_string(x,'Vagas'))

# Localidade
df['bairro'] = df['endereco'].apply(lambda x: x if isinstance(x,float) else x.split(' - ')[-2])
df['cidade'] = df['endereco'].apply(lambda x: x if isinstance(x,float) else x.split(' - ')[-1])

# Atributos do imóvel e condomínio
df['mobiliado'] = df['catacteristicas_imovel'].apply(lambda x: np.nan if isinstance(x,float) else 'Sim' if 'mobiliado' in x.lower() else 'Não')
df['piscina'] = df['detalhes_condominio'].apply(lambda x: np.nan if isinstance(x,float) else 'Sim' if 'piscina' in x.lower() else 'Não')
df['academia'] = df['detalhes_condominio'].apply(lambda x: np.nan if isinstance(x,float) else 'Sim' if 'academia' in x.lower() else 'Não')
df['sacada'] = df['catacteristicas_imovel'].apply(lambda x: np.nan if isinstance(x,float) else 'Sim' if 'sacada' in x.lower() else 'Não')
df['churrasqueira'] = df['detalhes_condominio'].apply(lambda x: np.nan if isinstance(x,float) else 'Sim' if 'churrasqueira' in x.lower() else 'Não')


# Save

In [21]:
columns_selected = [
 'titulo',
 'link',
 'endereco',
 'aluguel',
 'condominio',
 'iptu',
 'catacteristicas_imovel',
 'detalhes_condominio',
 'area',
 'quartos',
 'suites',
 'banheiros',
 'andar',
 'vagas_garagem',
 'bairro',
 'cidade',
 'mobiliado',
 'piscina',
 'academia',
 'sacada',
]

In [22]:
df[columns_selected].to_csv('../data/data_trusted/anuncios-cilar-trusted.csv', index = False)